In [ ]:
#Importing the necessary libraries required for further processing.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc,classification_report,confusion_matrix,accuracy_score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from scikitplot.metrics import plot_confusion_matrix,plot_precision_recall_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
# Importing the train and test dataset.
training_data = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')
test_data = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')

In [ ]:
#Fetching the top 5 records from the training dataset
training_data.head()

In [ ]:
#Displaying the shape of the training data
training_data.shape

In [ ]:
#Fetching the top 5 records from the test dataset
test_data.head()

In [ ]:
#Displaying the shape of the test data
test_data.shape

In [ ]:
#Fetching dimension, concise summary like type of data,non null values, memory used by data
print(f'Dimension of Train data {training_data.shape} \n Data feature informations')
print(training_data.info())
print('\n')
print(f'Dimension of Test data {test_data.shape} \n Data feature informations')
print(test_data.info())

In [ ]:
from IPython.display import display
pd.options.display.max_columns = None

In [ ]:
missing_train=training_data.isnull().sum()
percent_missing_train=training_data.isnull().sum()/training_data.isnull().count()*100
train=pd.concat([missing_train, percent_missing_train], axis=1, keys=['Total', 'Percent'])
trn = []
for col in training_data.columns:
        dtype_train = str(training_data[col].dtype)
        trn.append(dtype_train)
train['Data Type'] = trn
train_transpose=np.transpose(train)
train_transpose

In [ ]:
missing_test=test_data.isnull().sum()
percent_missing_test=test_data.isnull().sum()/test_data.isnull().count()*100
test=pd.concat([missing_test, percent_missing_test], axis=1, keys=['Total', 'Percent'])
tst = []
for col in test_data.columns:
        dtype_test = str(training_data[col].dtype)
        tst.append(dtype_test)
test['Data Type'] = tst
test_transpose=np.transpose(test)
test_transpose

In [ ]:
duplicate_train = training_data[training_data.duplicated()]
duplicate_train

In [ ]:
duplicate_test = test_data[test_data.duplicated()]
duplicate_test

In [ ]:
training_data.describe()

In [ ]:
test_data.describe()

In [ ]:
print("There are {} unique values in training dataset".format(training_data['target'].unique()))

In [ ]:
print(sns.countplot(training_data['target']))

In [ ]:
training_data['target'].value_counts(normalize=True)

In [ ]:
print("There are {}% target values with 1".format((training_data['target'].value_counts()[1]/training_data['target'].value_counts()[0])*100))

In [ ]:
feature_train = training_data[training_data.columns[2:]]
feature_test = test_data[test_data.columns[1:]]

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of mean values per column in the train set")
feature_train.mean().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of mean values per column in the  test set")
feature_test.mean().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.std().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.std().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.var().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.var().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.min().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.min().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.max().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.max().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.skew().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.skew().plot(kind='hist');

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the train set")
feature_train.kurtosis().plot(kind='hist');

plt.figure(figsize=(6,3))
plt.title("Distribution of SD values per column in the  test set")
feature_test.kurtosis().plot(kind='hist');

In [ ]:
corr = training_data.corr().abs()
corr[corr == 1] = 0
corr.unstack().sort_values(ascending=False).reset_index()
corr.head(10)

In [ ]:
#Finding skewness for each feature
skewList = []
for colName in feature_train.columns:
    if colName not in ['ID_code', 'target']:
        skewList.append([colName, abs(feature_train[colName].skew())])

skewList.sort(key=lambda x: x[1], reverse=True)

skewdf = pd.DataFrame.from_records(skewList, columns=['colName', 'Skewness'])

print(skewdf)

In [ ]:
skew = []
for col in feature_train.columns:
    skew.append([col, abs(feature_train[col].skew())])

In [ ]:
skew.sort(key=lambda x: x[1], reverse=True)

In [ ]:
a=pd.DataFrame.from_records(skew,columns=['Columns', 'Skewness'])
a

In [ ]:
max(a['Skewness'])

In [ ]:
min(a['Skewness'])

In [ ]:
# Boxplot Analysis
# Plot  features.
feature_train.iloc[:,:51].plot(kind='box', figsize=[14,7])
feature_train.iloc[:,51:101].plot(kind='box', figsize=[14,7])
feature_train.iloc[:,101:151].plot(kind='box', figsize=[14,7])
feature_train.iloc[:,151:].plot(kind='box', figsize=[14,7])

In [ ]:
# Finding outliers using IQR values
train_x = training_data.iloc[:, :]
IQR = train_x.quantile(.75) - train_x.quantile(.25)
print("Train.shape:",training_data.shape)
df_in = training_data[~((train_x < (train_x.quantile(.25) - 1.5 * IQR)) |(train_x > (train_x.quantile(.75) + 1.5 * IQR))).any(axis=1)]
df_out = training_data[((train_x < (train_x.quantile(.25) - 1.5 * IQR)) |(train_x > (train_x.quantile(.75) + 1.5 * IQR))).any(axis=1)]
print("df_in.shape:",df_in.shape)
print("df_out.shape:",df_out.shape)

In [ ]:
df_in

In [ ]:
print("df_in.target:\n", df_in['target'].value_counts())
print("df_out.target:\n", df_out['target'].value_counts())

In [ ]:
#Feature engineering
#remove column 'ID_code', which is just a index for the training dataset
#create training and testing data

X=training_data.drop(columns=['ID_code','target'],axis=1)
y=training_data['target']

test=test_data.drop(columns=['ID_code'],axis=1)

In [ ]:
print('X shape:',X.shape)
print('y shape:',y.shape)

In [ ]:

smote_sampling = SMOTE(random_state=42)

#Generating synthetic data points
X_smote,y_smote=smote_sampling.fit_resample(X,y)

In [ ]:
X_smote.shape

In [ ]:
y_smote.shape

In [ ]:
#Feature scaling
scaler = StandardScaler()
X_sc = scaler.fit_transform(X_smote)
test_sc = scaler.fit_transform(test)

In [ ]:
#Using Elbow method on scree plot to decide number of principal components to be used for PCA
pca = PCA().fit(X_smote)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.grid()

In [ ]:
pca = PCA(n_components=100, svd_solver='randomized').fit(X_smote)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.title('Santander unscaled Radomized PCA cumulative explained variance')
plt.grid()

In [ ]:
#Using PCA for dimensionality reduction
#pca=PCA(n_components=150)
pca1=PCA(copy=True, iterated_power='auto', n_components=150, random_state=None, svd_solver='auto', tol=0.0, whiten=False)

#Applying PCA on standardized train data
X_pca=pca1.fit_transform(X_sc)

#Applying PCA on standardized test data
test_data=pca1.fit_transform(test_sc)

In [ ]:
#Split the training data into training and validation data
X_train,X_val,y_train,y_val=train_test_split(X_pca,y_smote,test_size=0.25,random_state=42)

print('X_train shape:',X_train.shape)
print('X_val shape :',X_val.shape)
print('y_train shape:',y_train.shape)
print('y_val shape :',y_val.shape)

In [ ]:
logModel = LogisticRegression()

In [ ]:
#param_grid = [  
#    {
#    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
 #   'C' : np.logspace(-4, 4, 20),
#   'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
 #   'max_iter' : [100, 1000,2500, 5000]
 #   }
#]

In [ ]:
#clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
#best_clf = clf.fit(X_train,y_train)

In [ ]:
#print(best_clf.best_params_)

In [ ]:
#print (f'Accuracy - : {best_clf.score(X_train,y_train):.3f}')

In [ ]:
#Train data using logistic regression
logModel=LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
logModel.fit(X_train , y_train)

In [ ]:
#Accuracy of the model
lr_score=logModel.score(X_val,y_val)
print('Accuracy of the logistic regression model :',lr_score)

In [ ]:
#Cross validation prediction
cv_pred=cross_val_predict(logModel,X_val,y_val,cv=5)
#Cross validation score
cv_score=cross_val_score(logModel,X_val,y_val,cv=5)
print('cross_val_score :',np.average(cv_score))

In [ ]:
#Confusion matrix
cm=confusion_matrix(y_val,cv_pred)
#Plot the confusion matrix
plot_confusion_matrix(y_val,cv_pred,normalize=False,figsize=(12,6))

In [ ]:
#Classification report
scores=classification_report(y_val,cv_pred)
print(scores)

In [ ]:
#ROC score
roc_score=roc_auc_score(y_val,cv_pred)
print('ROC score :',roc_score)

#ROC_AUC curve
plt.figure()
false_positive_rate,recall,thresholds=roc_curve(y_val,cv_pred)
roc_auc=auc(false_positive_rate,recall)
plt.title('Receiver Operating Characteristics(ROC)')
plt.plot(false_positive_rate,recall,'b',label='ROC(area=%0.3f)' %roc_auc)
plt.legend()
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.ylabel('Recall(True Positive Rate)')
plt.xlabel('False Positive Rate')
plt.show()
print('AUC:',roc_auc)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT_model = DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, random_state=None,
            splitter='best')

In [ ]:
DT_model.fit(X_train , y_train)

In [ ]:
#Accuracy of the model
DT_score=DT_model.score(X_val,y_val)
print('Accuracy of the Decision Tree model :',DT_score)

In [ ]:
#Cross validation prediction
DT_cv_pred=cross_val_predict(DT_model,X_val,y_val,cv=5)

In [ ]:
#Cross validation score
DT_cv_score=cross_val_score(DT_model,X_val,y_val,cv=5)
print('cross_val_score :',np.average(DT_cv_score))

In [ ]:
#Confusion matrix
DT_cm=confusion_matrix(y_val,DT_cv_pred)
#Plot the confusion matrix
plot_confusion_matrix(y_val,DT_cv_pred,normalize=False,figsize=(8,4))

In [ ]:
#Classification report
scores=classification_report(y_val,DT_cv_pred)
print(scores)

In [ ]:
#ROC score
DT_roc_score=roc_auc_score(y_val,DT_cv_pred)
print('ROC score :',DT_roc_score)

In [ ]:
#ROC_AUC curve
plt.figure()
false_positive_rate,recall,thresholds=roc_curve(y_val,DT_cv_pred)
DT_roc_auc=auc(false_positive_rate,recall)
plt.title('Receiver Operating Characteristics(ROC)')
plt.plot(false_positive_rate,recall,'b',label='ROC(area=%0.3f)' %DT_roc_auc)
plt.legend()
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.ylabel('Recall(True Positive Rate)')
plt.xlabel('False Positive Rate')
plt.show()
print('AUC:',DT_roc_auc)

In [ ]:
RF_model = RandomForestClassifier()

In [ ]:
#random_search = {'criterion': ['entropy', 'gini'],
              #'max_depth': [2,10,20,30,40,50,None],
              # 'max_features': ['auto', 'sqrt'],
              # 'min_samples_leaf': [1,4,8],
              # 'min_samples_split': [2,5,10],
              # 'n_estimators': [20,40,60,80,100],
              #  'bootstrap': [True, False]}

In [ ]:
#clf = GridSearchCV(RF_model, param_grid = random_search, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
#best_clf = clf.fit(X_train,y_train)

In [ ]:
#print(best_clf.best_params_)

In [ ]:
RF_model=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
RF_model.fit(X_train , y_train)

In [ ]:
#Accuracy of the model
RF_score=RF_model.score(X_val,y_val)
print('Accuracy of the Random Forest model :',RF_score)

In [ ]:
#Cross validation prediction
RF_cv_pred=cross_val_predict(RF_model,X_val,y_val,cv=5)
#Cross validation score
RF_cv_score=cross_val_score(RF_model,X_val,y_val,cv=5)
print('cross_val_score :',np.average(RF_cv_score))

In [ ]:
#Confusion matrix
RF_cm=confusion_matrix(y_val,RF_cv_pred)
#Plot the confusion matrix
plot_confusion_matrix(y_val,RF_cv_pred,normalize=False,figsize=(8,4))

In [ ]:
#Classification report
scores=classification_report(y_val,RF_cv_pred)
print(scores)

In [ ]:
#ROC score
RF_roc_score=roc_auc_score(y_val,RF_cv_pred)
print('ROC score :',RF_roc_score)

#ROC_AUC curve
plt.figure()
false_positive_rate,recall,thresholds=roc_curve(y_val,RF_cv_pred)
RF_roc_auc=auc(false_positive_rate,recall)
plt.title('Receiver Operating Characteristics(ROC)')
plt.plot(false_positive_rate,recall,'b',label='ROC(area=%0.3f)' %RF_roc_auc)
plt.legend()
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.ylabel('Recall(True Positive Rate)')
plt.xlabel('False Positive Rate')
plt.show()
print('AUC:',RF_roc_auc)

In [ ]:
import lightgbm as lgb

In [ ]:
params = {'objective' : "binary", 
                'learning_rate' : 0.01,
               'boost':"gbdt",
                'bagging_fraction' : 0.4,
               'min_data_in_leaf' : 80,
               'metric':"auc",
               'boost_from_average':"false",
               'num_threads':8,               
               'num_leaves' : 13,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 0.05,
               'bagging_freq' : 5,               
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)

In [ ]:
lightGBM_model = lgb.train(params, d_train, 100)

In [ ]:
lightGBM_pred=lightGBM_model.predict(X_val)
#convert into binary values
for i in range(0,len(lightGBM_pred)):
    if lightGBM_pred[i]>=.5:       # setting threshold to .5
       lightGBM_pred[i]=1
    else:  
       lightGBM_pred[i]=0

In [ ]:
#Accuracy

lightGBM_accuracy=accuracy_score(lightGBM_pred,y_val)
print("Accuracy using LightGBM is : ",lightGBM_accuracy)

In [ ]:
#Confusion matrix

lightGBM_cm = confusion_matrix(y_val, lightGBM_pred)

#Plot the confusion matrix
plot_confusion_matrix(y_val,lightGBM_pred,normalize=False,figsize=(8,4))

In [ ]:
#Classification report
scores=classification_report(y_val,lightGBM_pred)
print(scores)

In [ ]:
#ROC score
LightGBM_roc_score=roc_auc_score(y_val,lightGBM_pred)
print('ROC score :',LightGBM_roc_score)

#ROC_AUC curve
plt.figure()
false_positive_rate,recall,thresholds=roc_curve(y_val,lightGBM_pred)
roc_auc=auc(false_positive_rate,recall)
plt.title('Receiver Operating Characteristics(ROC)')
plt.plot(false_positive_rate,recall,'b',label='ROC(area=%0.3f)' %roc_auc)
plt.legend()
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.ylabel('Recall(True Positive Rate)')
plt.xlabel('False Positive Rate')
plt.show()
print('AUC:',roc_auc)

In [ ]:
#Predicting the model performance on test data

RF_test_pred=RF_model.predict(test_data)
print(RF_test_pred)

In [ ]:
test_data

In [ ]:
test_data.drop(test_data.columns[1:], axis=1, inplace=True)

In [ ]:
my_submission = pd.DataFrame({'ID_code': test_data.ID_code, 'target': RF_test_pred})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission